# Notebook description

This notebook is for developing the quadratic integrate-and-fire (QIF) neuron.

# The QIF dynamics

The QIF neuron dynamics follows

\begin{equation*}
 \tau \dot{x} = -x + \frac{x^2}{2} + u
\end{equation*}

where $x$ is the state of the neuron, $\tau$ is the membrane time constant, and $u$ is the neuron input. The firing rate is given by

\begin{equation*}
f = \frac{1}{t_{ref}+t_{spk}}
\end{equation*}

where $t_{ref}$ is the refractory period and $t_{spk}$ is the interval from the end of the refractory period to when the neuron spikes. A neuron is considered to have spiked when its state reaches $\infty$. The cool thing about the QIF neuron is that it can reach $\infty$ in finite time given the appropriate initial condition or input. To find the inter-spike interval, we integrate from the end of the refractory period to the spike time. Assuming the refractory period ended at $t=0$,

\begin{align*}
\tau \frac{dx}{dt} &= -x + \frac{x^2}{2} + u \\
dt &= \frac{\tau}{-x + \frac{x^2}{2} + u}dx \\
\int_0^{t_{spk}}dt &= \int_0^\infty\frac{\tau}{-x + \frac{x^2}{2} + u}dx \\
t_{spk} &= \frac{\tau  \left(\pi + 2 \cot ^{-1}\left(\sqrt{2u-1}\right)\right)}{\sqrt{2u-1}} 
    & \mathrm{for}\;u\ge0.5 \\
\end{align*}

In fact, it is possible to solve the QIF neuron dynamics analytically. The actual solution to the QIF neuron is given by [???]

This solution shows that the QIF neuron actually resets to $-\infty$ and begins integrating again

Unfortunately in software, we can't integrate to $\infty$. Instead we set a threshold $x_t$ at which point the neuron is declared to have spiked, and when a neuron reaches threshold, we reset the neuron to $0$. In this way, the software QIF neuron spikes just like the LIF neuron. This conscession changes the spike rate of the neuron

\begin{align*}
t_{spk} &= \int_0^{x_t}\frac{\tau}{-x + \frac{x^2}{2} + u}dx \\
 &= \frac{2\tau\left(\cot^{-1}\left(\frac{\sqrt{2u-1}}{x_t-1}\right)+
    \cot^{-1}\left(\sqrt{2u-1}\right)\right)}
     {\sqrt{2u-1}}
     & \mathrm{for}\;u\ge0.5 \\
\end{align*}

## open questions

How long until a QIF neuron spikes from a nonzero starting voltage with a given input that will cause the neuron to spike? Actually at threshold with no input ($x=2$, $u=0$), the neuron will not spike. But any slight bump up from $x=2$ should make it spike. Can I have it spike at arbitrary time up to $\infty$?

In [1]:
%matplotlib inline
import numpy as np
from numpy.random import lognormal
import matplotlib.pyplot as plt

import nengo
from nengo.builder import Builder
from nengo.builder.signal import Signal
from nengo.builder.neurons import SimNeurons

In [ ]:
class QIFRate(nengo.neurons.NeuronType):  # Neuron types must subclass `nengo.Neurons`
    """A rate-based quadratic integrate-and-fire neuron model"""   
    tau_rc = NumberParam(low=0, low_open=True)
    tau_ref = NumberParam(low=0)
    probeable = ['rates']
    threshold = 10.0  # Use this as the threshold for saying if a neuron has spiked

    def __init__(self, tau_rc, tau_ref):
        """You should not modify this function"""
        self.tau = tau_rc    # time constant (can contain mismatch)
        self.tau_ref = tau_ref
        
    def gain_bias(self, max_rates, intercepts):
        """Return gain and bias given maximum firing rate and x-intercept.
        
        For our QIF implementation, we rely on the intrinsic variability
        of the transistors to provide the heterogeneity in the tuning curves
        and so set the gain to 1 and bias to 0.
        
        You should not modify this function
        """
        gain = np.ones(max_rates.shape)
        bias = np.zeros(max_rates.shape)
        return gain, bias

    def step_math(self, dt, J, output):
        """Compute the output rates in Hz for input current
        
        Parameters
        ----------
        dt : float
            simulation time step. Unused in this function.
        J : N-long 1D array of floats (during simulation)
            input current to N neurons 
            or
            (k x N) 2D array of floats (when using the tuning_curves function)
            k input currents to N neurons
        output : N-long, 1D array of floats
            spike rate of neurons
        """
        def clip_sqrt(x):
            x[x<0] = 0.
            return np.sqrt(x)
            
        _inp = self.kbg * self.ibg + self.kin * J
        x = 2. * self.km * _inp - 1.
        idx = x > 0.
        num = clip_sqrt(x)
        den = clip_sqrt(x)
        atan_x_num = QIFRate.threshold*self.km - 1 + np.zeros(den.shape)
        den[idx] = np.arctan(atan_x_num[idx]/den[idx]) + np.arctan(1./den[idx])
        den *= 2. * self.tau
        output[idx] = num[idx]/den[idx]
        output[~idx] = 0.


# Register the QIFRate model with Nengo
# You should not modify anything below here in this cell
@Builder.register(QIFRate)
def build_qifrate(model ,qifrate, neurons):
    model.add_op(SimNeurons(
        neurons=qifrate,
        J=model.sig[neurons]['in'],
        output=model.sig[neurons]['out']))

In [ ]:
class QIF(QIF):
    """A spiking quadratic integrate-and-fire neuron model"""   
    def step_math(self, dt, J, spiked, voltage):
        """Integrate the QIF model one step forward in time using Heun's method

        Parameters
        ----------
        dt : float
            simulation time step
        J : array of floats
            input current to neurons
        spiked : array of 0 or 1/dt's
            indicates which neurons has spiked during time step
        voltage : array of floats
            voltage of neurons
        """
        ########### Your code here ###########
        vv = voltage
        dV = dt / self.tau * (-vv + self.km*vv**2/2 + self.kbg*self.ibg + self.kin*J)
        v = vv + dV
        dV2 = dt / self.tau * (-v + self.km*v**2/2 + self.kbg*self.ibg + self.kin*J)
        dVavg = (dV+dV2)/2.
        v = vv + dVavg
        
        spiked[:] = np.where(v > QIFRate.threshold, 1.0/dt, 0.0)

        v[spiked > 0] = 0.
        voltage[:] = v

# Register the QIF_heun model with Nengo
# You should not have to modify this
@Builder.register(QIF_heun)
def build_qif_heun(model, qif_heun, neurons):
    model.sig[neurons]['voltage'] = Signal(
        np.zeros(neurons.size_in), name="%s.voltage" % neurons)
    model.add_op(SimNeurons(
        neurons=qif_heun,
        J=model.sig[neurons]['in'],
        output=model.sig[neurons]['out'],
        states=[model.sig[neurons]['voltage']]))

In [ ]:
############### Your Code Here ##############
idx = 2
dts = [.01, .001, .0001]
T = .2
net = nengo.Network()
with net:
    stim = nengo.Node(1.)
    neuron_type = QIF(
        ptau_list[idx] * tau, a_list[idx], b_list[idx], c_list[idx], ibg)
    ens = nengo.Ensemble(1, 1, neuron_type=neuron_type,
                       encoders=np.ones((1, 1)))
    nengo.Connection(stim, ens, synapse=0)
    probe_v = nengo.Probe(ens.neurons, 'voltage')
fig, axs = plt.subplots(ncols=3, figsize=(15,4))
for idx, dt in enumerate(dts):
    sim = nengo.Simulator(net, dt=dt)
    sim.run(T, progress_bar=False)
    t = sim.trange()
    axs[idx].plot(sim.trange(), sim.data[probe_v])
    axs[idx].set_ylabel('voltage', fontsize=14)
    axs[idx].set_xlabel(r'$t$', fontsize=20)

In [ ]:
def check_tuning_curve_accuracy(neuron_model):
    """Takes in a neuron model (e.g. QIF) and checks the accuracy"""
############### Your Code Here ##############
    xs = np.linspace(-1., 1., 30)
    N = 5
    idx = np.random.choice(np.arange(num_neurons), N, replace=False)
    T = 1.
    dt = .0001

    net = nengo.Network()
    with net:
        stim = nengo.Node(0.)
        encoders = np.random.choice([-1, 1], N, replace=True).reshape((N, 1))
        neuron_type = neuron_model(ptau_list[idx] * tau, a_list[idx], b_list[idx], c_list[idx], ibg)
        ens = nengo.Ensemble(N, 1, encoders=encoders, neuron_type=neuron_type)
        nengo.Connection(stim, ens, synapse=0.)
        probe_s = nengo.Probe(ens.neurons, 'spikes')
        probe_v = nengo.Probe(ens.neurons, 'voltage')

    a_hat = np.zeros((len(xs), N))
    for x_idx, x in enumerate(xs):
        stim.output = x
        sim = nengo.Simulator(net, dt=dt)
        sim.run(T, progress_bar=False)
        t = sim.trange()
        spike_dat = sim.data[probe_s]
        for n in xrange(N):
            spk_times = t[np.nonzero(spike_dat[:, n])[0]]
            if len(spk_times) == 0:
                a_hat[x_idx, n] = 0
            elif len(spk_times) == 1:
                a_hat[x_idx, n] = 1./T
            else:
                isi = np.diff(spk_times)
                a_hat[x_idx, n] = 1./np.mean(isi)

    sim = nengo.Simulator(net)
    _, a = tuning_curves(ens, sim, inputs=xs.reshape((-1,1)))

    a_error = a - a_hat

    cc = ['r', 'g', 'b', 'c', 'm']
    fig, axs = plt.subplots(ncols=2, figsize=(15,5))
    axs[0].set_color_cycle(cc)
    axs[0].plot(xs, a)
    axs[0].plot(xs, a_hat, 'o', ms=6)
    axs[0].set_xlabel(r'$x$', fontsize=20)
    axs[0].set_ylabel(r'$a,\ \hat{a}$', fontsize=20)
    axs[1].set_color_cycle(cc)
    axs[1].plot(xs, a_error)
    axs[1].set_xlabel(r'$x$', fontsize=20)
    axs[1].set_ylabel(r'$a-\hat{a}$', fontsize=20);